In [2]:
import pandas as pd
import numpy as np

users = pd.read_csv(r'C:\Users\Dell\Downloads\assignment3\takehome_users.csv', encoding='latin-1',parse_dates=True)
engage = pd.read_csv(r'C:\Users\Dell\Downloads\assignment3\takehome_user_engagement.csv', parse_dates=True)
users
engage

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1
...,...,...,...
207912,2013-09-06 06:14:15,11996,1
207913,2013-01-15 18:28:37,11997,1
207914,2014-04-27 12:45:16,11998,1
207915,2012-06-02 11:55:59,11999,1


In [ ]:
import datetime

#set the time_stamp to datetime and the set it as the index
engage.time_stamp = pd.to_datetime(engage.time_stamp)
engage = engage.set_index('time_stamp', drop= True)


from datetime import timedelta

def label_adopted(x):    
    "takes a users input and returns whether or not they have been active within any 7-day period"
    df_temp = engage.loc[engage['user_id'] == x] #select out rows of this user
    df_temp = df_temp.resample('D').mean().dropna() #resample to show if active in a day. .mean() is just of 1
    adopted = 0
    for i in range(len(df_temp)-2): #loop over active days till the second to last day
        if df_temp.index[i + 2] - df_temp.index[i] <= timedelta(days=7): # difference between every 1st and 3rd day
            adopted = 1
            break
        else:
            adopted = 0
    return adopted

#apply to user df to label users as adopted=true 
users['adopted_user'] = users['object_id'].apply(label_adopted)


print(sum(users['adopted_user']))
print(sum(users.adopted_user)/len(users.adopted_user))

In [ ]:
More data wrangling. Convert all time stamps to datetime objects.

# set creation_time
users.creation_time = pd.to_datetime(users.creation_time)
#last_session_creation_time is a unix time stamp, so use map to convert to datetime
users['last_session_creation_time'] = users['last_session_creation_time'].map(lambda data: 
                                    datetime.datetime.fromtimestamp(int(data)).strftime('%Y-%m-%d %H:%M:%S'),
                                                                              na_action='ignore')
Subtract thecreation_time from the last_session_creation_time to create a feature that combines the two in a meaningful way. This will give us feature usage_length that basically indicates how long a user has been active.

#now set that to datetime
users['last_session_creation_time'] = pd.to_datetime(users['last_session_creation_time'])
#subtract to find time active
users['usage_length'] = users['last_session_creation_time'] - users['creation_time']
#lets settle for seconds instead of days to make the time differences more distinct
users['usage_length'] = [x.total_seconds() for x in users['usage_length']]
Looking good so far, but we can also use email domain as a feature as well. There are only a few main ones, so I will label the less popular domains as other.

users['email_provider'] = [x.split('@')[1] for x in users.email]#select out the domain
top_emails = users.email_provider.value_counts().index[:6]
#label anything not in the top 5 as other
users['email_provider'] = [x if x in top_emails else 'other' for x in users.email_provider]

In [ ]:
The same can be done with users, but first the emtpy values must be filled to be categorized.

users.invited_by_user_id = users.invited_by_user_id.fillna(0)
Very quickly, remove the columns containing features that won't be useful for analysis. This includes object_id creation_time name email and last_session_creation_time.

feature_df = users.iloc[:,4:]
feature_df = feature_df.drop('last_session_creation_time', axis=1)
feature_df['usage_length'] = feature_df['usage_length'].fillna(0)
Using sklearn.preprocessing.LabelEncoder we can easily engineer categorical labels for each categorical feature.

from sklearn.preprocessing import LabelEncoder